<a href="https://colab.research.google.com/github/karthikpuranik11/Hope-Speech-Detection-/blob/main/English/CharacterBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers==3.3.1

     |████████████████████████████████| 1.1MB 13.0MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
     |████████████████████████████████| 1.1MB 46.6MB/s 
     |████████████████████████████████| 3.0MB 50.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ab91fd3bff7738fc49c92a7f21a2c46ba50294eed946885742c21b0f1b7447c5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!git clone https://github.com/helboukkouri/character-bert.git

Cloning into 'character-bert'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 51 (delta 12), reused 31 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [4]:
cd /content/character-bert/ 

/content/character-bert


In [5]:
!python download.py --model='general_character_bert'

04/01/2021 09:07:16 - INFO - download.py -   Downloading general_character_bert model (~200MB tar.xz archive)
04/01/2021 09:07:20 - INFO - download.py -   Extracting model from archive (~420MB folder)
04/01/2021 09:07:34 - INFO - download.py -   Removing archive
04/01/2021 09:07:34 - INFO - download.py -   Done.


In [6]:
import numpy as np
import pandas as pd
import time
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification
from torch.utils.data import Dataset,DataLoader,TensorDataset,RandomSampler,SequentialSampler
from transformers import BertConfig,BertTokenizer,BertForSequenceClassification,AdamW
from modeling.character_bert import CharacterBertModel
from utils.character_cnn import CharacterIndexer
from tqdm import trange,tqdm
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [7]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [8]:
config = BertConfig.from_pretrained('bert-base-uncased',num_labels=3)
model = BertForSequenceClassification(config= config)
character_bert_model = CharacterBertModel.from_pretrained('./pretrained-models/general_character_bert/')
model.bert = character_bert_model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.cuda()
device = 'cuda'

In [9]:
train=pd.read_csv('/content/english_hope_train.csv', header=None, names=['tweets','label','nan'], sep="\t")
#train['labels']=LabelEncoder().fit_transform(train['label'])
train=train.drop(columns=['nan'])
train.label.value_counts()

Non_hope_speech    20778
Hope_speech         1962
not-English           22
Name: label, dtype: int64

In [10]:
val=pd.read_csv('/content/english_hope_dev.csv', header=None, names=['tweets','label','nan'], sep="\t")
#val['labels']=LabelEncoder().fit_transform(val['label'])
val=val.drop(columns=['nan'])
val.label.value_counts()

Non_hope_speech    2569
Hope_speech         272
not-English           2
Name: label, dtype: int64

In [16]:
def punc(df):
    df['tweets'] = df['tweets'].str.replace(r"[+/#@&*$%:0-9]",'')
    print(df)
punc(train)
punc(val)

                                                  tweets            label
0      these tiktoks radiate gay chaotic energy and i...  Non_hope_speech
1      Champions Again He got killed for using false ...  Non_hope_speech
2                   It's not that all lives don't matter  Non_hope_speech
3      Is it really that difficult to understand? Bla...  Non_hope_speech
4      Whenever we say black isn't that racists?  Why...  Non_hope_speech
...                                                  ...              ...
22757  It's a load of bollocks every life matters sim...  Non_hope_speech
22758  no say it because all lives matter! deku would...  Non_hope_speech
22759                          God says her life matters  Non_hope_speech
22760  This video is just shit. A bunch of whiny ass ...  Non_hope_speech
22761  Mc Fortnut she did  months ago in west church ...  Non_hope_speech

[22762 rows x 2 columns]
                                                 tweets            label
0          Th

In [21]:
frames = [train,val]
df = pd.concat(frames)

In [27]:
len(train),len(val)
print(round(2843/(22762+2843),4))

0.111


In [29]:
df['label'],uniq = pd.factorize(df['label'])
X = df['tweets'].tolist()
tokenized = [tokenizer.basic_tokenizer.tokenize(text) for text in X]
indexer = CharacterIndexer()
input_tensor = indexer.as_padded_tensor(tokenized)
X_train,X_test,y_train,y_test = train_test_split(input_tensor,df['label'].tolist(),
                                                 test_size=0.111,random_state=42)


In [30]:
batch_size = 32
y_train,y_test = torch.tensor(y_train),torch.tensor(y_test)
train_data = TensorDataset(X_train,y_train)
train_dataloader = DataLoader(train_data,batch_size = batch_size)
val_data = TensorDataset(X_test,y_test)
val_dataloader = DataLoader(val_data,batch_size=batch_size)

In [31]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5)

In [32]:
train_loss_set = []
epochs = 3
from collections import defaultdict
history = defaultdict(list)

for _ in trange(epochs, desc="Epoch"):
  start_time = time.time()
  model.train()
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  for step, batch in enumerate(train_dataloader):

    batch = tuple(t.to(device) for t in batch)
  
    b_input_ids, b_labels = batch

    optimizer.zero_grad()
  
    outputs = model(b_input_ids)[0]
    
    loss = loss_fn(outputs,b_labels)
    train_loss_set.append(loss.item())    
    
    loss.backward()

    optimizer.step()
    
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  end_time = time.time()

  #epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  print(epoch_time(start_time,end_time))
  print("\nTrain loss: {}".format(tr_loss/nb_tr_steps))
torch.save(model.state_dict(),'general-character-bert.bin')

Epoch:  33%|███▎      | 1/3 [22:18<44:37, 1338.82s/it]

(22, 18)

Train loss: 0.22667165134845071


Epoch:  67%|██████▋   | 2/3 [44:53<22:23, 1343.54s/it]

(22, 34)

Train loss: 0.16001366102856093


Epoch: 100%|██████████| 3/3 [1:07:26<00:00, 1348.94s/it]

(22, 33)

Train loss: 0.1152677056124549


In [33]:
preds = []
with torch.no_grad():
  correct = 0
  total = 0
  for i, batch in enumerate(val_dataloader):
    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_labels = batch
    
    outputs = model(b_input_ids)[0]
    # print (outputs)
    prediction = torch.argmax(outputs,dim=1)
    preds.append(prediction)
    total += b_labels.size(0)
    correct+=(prediction==b_labels).sum().item()

In [34]:
final_preds = []
for tensor in preds:
  for pred in tensor:
    final_preds.append(int(pred))

In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2604
           1       0.54      0.54      0.54       233
           2       0.00      0.00      0.00         6

    accuracy                           0.92      2843
   macro avg       0.50      0.50      0.50      2843
weighted avg       0.92      0.92      0.92      2843



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
